In [1]:
# pip install jupyter_argparser

In [7]:
# import the necessary packages
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--dataset", required=True,
# 	help="path to input directory of faces + images")
# ap.add_argument("-e", "--embeddings", required=True,
# 	help="path to output serialized db of facial embeddings")
# ap.add_argument("-d", "--detector", required=True,
# 	help="path to OpenCV's deep learning face detector")
# ap.add_argument("-m", "--embedding-model", required=True,
# 	help="path to OpenCV's deep learning face embedding model")
# ap.add_argument("-c", "--confidence", type=float, default=0.5,
# 	help="minimum probability to filter weak detections")
# # args = vars(ap.parse_args())
# args = parser.parse_args(args=['--dataset', '10', '--embeddings', '10'])

class Args:
    dataset = 'lib_small/'
    embeddings = 'output/embeddings.pickle'
    detector = 'face_detection_model/'
    embedding_model = 'openface.nn4.small2.v1.t7'
    confidence = 0.8
    
  

args = Args()
args.embedding_model

'openface.nn4.small2.v1.t7'

In [8]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join([args.detector, "deploy.prototxt.txt"])
modelPath = os.path.sep.join([args.detector,
	"res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(args.embedding_model)

[INFO] loading face detector...
[INFO] loading face recognizer...


In [9]:
# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images(args.dataset))


[INFO] quantifying faces...


In [14]:
# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []
# initialize the total number of faces processed
total = 0

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    print("[INFO] processing image {}/{}".format(i + 1,len(imagePaths)))
    name = imagePath.split(os.path.sep)[-2]
    name = name.split("/a")
    print(imagePath.split(os.path.sep)[-2])
    # load the image, resize it to have a width of 600 pixels (while
    # maintaining the aspect ratio), and then grab the image
    # dimensions
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=600)
#     cv2.imshow(name, image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    (h, w) = image.shape[:2]
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)
    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    # ensure at least one face was found
    if len(detections) > 0:
        # we're making the assumption that each image has only ONE
        # face, so find the bounding box with the largest probability
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]
        # ensure that the detection with the largest probability also
        # means our minimum probability test (thus helping filter out
        # weak detections)
        if confidence > args.confidence:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # extract the face ROI and grab the ROI dimensions
            face = image[startY:endY, startX:endX]
#             cv2.imshow(name, face)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
            (fH, fW) = face.shape[:2]
            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue  
            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()
#             print(vec, vec.flatten())
            # add the name of the person + corresponding face
            # embedding to their respective lists
            knownNames.append(name)
            knownEmbeddings.append(vec.flatten())
#             print(total, knownEmbeddings)
            total += 1

[INFO] processing image 1/85
lib_small/Albena
[INFO] processing image 2/85
lib_small/Albena
[INFO] processing image 3/85
lib_small/Albena
[INFO] processing image 4/85
lib_small/Albena
[INFO] processing image 5/85
lib_small/Albena
[INFO] processing image 6/85
lib_small/Albena
[INFO] processing image 7/85
lib_small/Albena
[INFO] processing image 8/85
lib_small/Albena
[INFO] processing image 9/85
lib_small/Albena
[INFO] processing image 10/85
lib_small/Albena
[INFO] processing image 11/85
lib_small/Albena
[INFO] processing image 12/85
lib_small/Albena
[INFO] processing image 13/85
lib_small/Albena
[INFO] processing image 14/85
lib_small/Albena
[INFO] processing image 15/85
lib_small/Albena
[INFO] processing image 16/85
lib_small/AlexandreF
[INFO] processing image 17/85
lib_small/AlexandreF
[INFO] processing image 18/85
lib_small/AlexandreF
[INFO] processing image 19/85
lib_small/AlexandreF
[INFO] processing image 20/85
lib_small/AlexandreF
[INFO] processing image 21/85
lib_small/Alexandre

In [11]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open(args.embeddings, "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing 85 encodings...


In [12]:
print(knownNames)

['lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/Albena', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/AlexandreF', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Dorian', 'lib_small/Erwan', 'lib_small/Erwan', 'lib_small/Erwan', 'lib_small/Erwan', 'li